In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time 
import mediapipe as mp

In [2]:
# Initializing model
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [3]:
def mediapipe_detection(image, model):
    """Detects mediapipe landmarks and returns as results"""
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [5]:
def draw_styled_landmarks(image, results):
    """Draws landmarks to the image"""
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1))
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

In [6]:
def extract_keypoints(results):
    """Collecting data
    Takes landmarks and saves it as numpy array"""
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [138]:
DATA_PATH = os.path.join('MP_Data')
# path_action_types = 'types of actions'

# Actions
actions = np.array([action for action in os.listdir(DATA_PATH)])

# Number of videos per action
no_sequences = 50

# Number of frames per video
sequence_length = 30
actions

array(['climb', 'cry', 'drink', 'eat', 'fall', 'give', 'jump', 'kick',
       'look', 'push', 'run', 'sit', 'sleep', 'wait', 'walk', 'wash'],
      dtype='<U5')

In [139]:
# Creating Directories for saving Dataset
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [210]:
# Collecting dataset
# AKA: 54 min of pain and family members asking "why u repeatedly touching your eye???"

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    for i, action in enumerate(actions):
        
#         Shows the tutorial image first for 2.5 sec
        tutorial_image = os.path.join(path_action_types, action+'.png')
        tutorial_image = cv2.imread(tutorial_image, cv2.IMREAD_COLOR)
        tutorial_image = cv2.resize(tutorial_image, (450, 360))
        h, w, _ = tutorial_image.shape

        cv2.imshow('OpenCV Feed', tutorial_image)
        cv2.waitKey(2500)

#         For every frame
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):
                
#                 Draw landmarks
                ret, frame = cap.read()
                image, results = mediapipe_detection(frame, holistic)
                draw_styled_landmarks(image, results)
                
#                 Getting tutorial image
                tutorial_image = os.path.join(path_action_types, action+'.png')
                tutorial_image = cv2.imread(tutorial_image, cv2.IMREAD_COLOR)
                tutorial_image = cv2.resize(tutorial_image, (150, 120))
                h, w, _ = tutorial_image.shape
            
                image[360:h+360, 490:w+490] = tutorial_image
                
                
#                 if first frame, then print -> Get ready, STARTING COLLECTING                
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    
                    cv2.putText(image, f'({i}) {action}: {sequence}', (40, 25),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
                    
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(1400)
                    
#                     Else just print type of action and number of cadr
                else:
                    cv2.putText(image, f'({i}) {action}: {sequence}', (40, 25),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
                    
                    cv2.imshow('OpenCV Feed', image)
                    
#                 Extract keypoints
                keypoints = extract_keypoints(results)
    
#                 and save the numpy array
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

#                 break if interrupt of pressing 'q'
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
#     close windows and camera
    cap.release()
    cv2.destroyAllWindows()

SyntaxError: invalid syntax (4050059373.py, line 63)

In [140]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [141]:
label_map = {label:num for num, label in enumerate(actions)}

In [142]:
sequences, labels = [], []

# Get dataset back
for action in actions:
    for sequence in range(no_sequences):
        window = []
        
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), f"{frame_num}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [143]:
np.array(sequences).shape

(800, 30, 1662)

In [144]:
np.array(labels).shape

(800,)

In [145]:
X = np.array(sequences)

In [146]:
# long explanation: just look lines: 212 and 147
y = to_categorical(labels).astype(int)

In [212]:
# y before
y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,


In [147]:
# y after
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [148]:
# Divide into train, test, and validation parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, shuffle=True)

In [149]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.05, shuffle=True)

In [150]:
y_train.shape

(722, 16)

In [151]:
X_train.shape

(722, 30, 1662)

In [152]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard

In [153]:
from tensorflow.keras.optimizers import Adam

In [154]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [155]:
model = Sequential()
# Learning basic stuff about LSTM might be helpful
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
# To avoid overfitting Dropout added
model.add(Dropout(0.4))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [156]:
model.compile(optimizer=Adam(learning_rate=1e-06), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [157]:
history = model.fit(X_train, y_train, epochs=800, callbacks=[tb_callback], validation_data=(X_valid, y_valid))

Epoch 1/800
23/23 [==============================] - 6s 146ms/step - loss: 2.7672 - categorical_accuracy: 0.0817 - val_loss: 2.7708 - val_categorical_accuracy: 0.0789
Epoch 2/800
23/23 [==============================] - 3s 127ms/step - loss: 2.7678 - categorical_accuracy: 0.0817 - val_loss: 2.7699 - val_categorical_accuracy: 0.0789
Epoch 3/800
23/23 [==============================] - 3s 129ms/step - loss: 2.7696 - categorical_accuracy: 0.0720 - val_loss: 2.7693 - val_categorical_accuracy: 0.0789
Epoch 4/800
23/23 [==============================] - 3s 128ms/step - loss: 2.7682 - categorical_accuracy: 0.0776 - val_loss: 2.7686 - val_categorical_accuracy: 0.1053
Epoch 5/800
23/23 [==============================] - 3s 122ms/step - loss: 2.7672 - categorical_accuracy: 0.0665 - val_loss: 2.7676 - val_categorical_accuracy: 0.1053
Epoch 6/800
23/23 [==============================] - 3s 127ms/step - loss: 2.7647 - categorical_accuracy: 0.0970 - val_loss: 2.7667 - val_categorical_accuracy: 0.105

Epoch 50/800
23/23 [==============================] - 3s 119ms/step - loss: 2.6563 - categorical_accuracy: 0.1066 - val_loss: 2.7231 - val_categorical_accuracy: 0.1316
Epoch 51/800
23/23 [==============================] - 3s 120ms/step - loss: 2.6704 - categorical_accuracy: 0.0817 - val_loss: 2.7189 - val_categorical_accuracy: 0.0789
Epoch 52/800
23/23 [==============================] - 3s 120ms/step - loss: 2.6554 - categorical_accuracy: 0.0900 - val_loss: 2.7192 - val_categorical_accuracy: 0.1053
Epoch 53/800
23/23 [==============================] - 3s 122ms/step - loss: 2.6436 - categorical_accuracy: 0.1136 - val_loss: 2.7198 - val_categorical_accuracy: 0.1053
Epoch 54/800
23/23 [==============================] - 3s 124ms/step - loss: 2.6574 - categorical_accuracy: 0.0970 - val_loss: 2.7166 - val_categorical_accuracy: 0.0789
Epoch 55/800
23/23 [==============================] - 3s 119ms/step - loss: 2.6362 - categorical_accuracy: 0.1053 - val_loss: 2.7143 - val_categorical_accuracy:

Epoch 99/800
23/23 [==============================] - 3s 127ms/step - loss: 2.5351 - categorical_accuracy: 0.1510 - val_loss: 2.6529 - val_categorical_accuracy: 0.2368
Epoch 100/800
23/23 [==============================] - 3s 133ms/step - loss: 2.5679 - categorical_accuracy: 0.1288 - val_loss: 2.6494 - val_categorical_accuracy: 0.2368
Epoch 101/800
23/23 [==============================] - 3s 135ms/step - loss: 2.5550 - categorical_accuracy: 0.1371 - val_loss: 2.6490 - val_categorical_accuracy: 0.2368
Epoch 102/800
23/23 [==============================] - 3s 131ms/step - loss: 2.5348 - categorical_accuracy: 0.1399 - val_loss: 2.6477 - val_categorical_accuracy: 0.2368
Epoch 103/800
23/23 [==============================] - 3s 121ms/step - loss: 2.5405 - categorical_accuracy: 0.1593 - val_loss: 2.6474 - val_categorical_accuracy: 0.2368
Epoch 104/800
23/23 [==============================] - 3s 123ms/step - loss: 2.5337 - categorical_accuracy: 0.1413 - val_loss: 2.6483 - val_categorical_accu

23/23 [==============================] - 3s 113ms/step - loss: 2.4624 - categorical_accuracy: 0.2064 - val_loss: 2.5702 - val_categorical_accuracy: 0.1842
Epoch 148/800
23/23 [==============================] - 3s 129ms/step - loss: 2.4629 - categorical_accuracy: 0.1939 - val_loss: 2.5692 - val_categorical_accuracy: 0.1842
Epoch 149/800
23/23 [==============================] - 3s 126ms/step - loss: 2.4396 - categorical_accuracy: 0.2105 - val_loss: 2.5672 - val_categorical_accuracy: 0.1842
Epoch 150/800
23/23 [==============================] - 3s 123ms/step - loss: 2.4522 - categorical_accuracy: 0.2105 - val_loss: 2.5654 - val_categorical_accuracy: 0.2105
Epoch 151/800
23/23 [==============================] - 3s 137ms/step - loss: 2.4453 - categorical_accuracy: 0.2036 - val_loss: 2.5637 - val_categorical_accuracy: 0.2105
Epoch 152/800
23/23 [==============================] - 3s 134ms/step - loss: 2.4560 - categorical_accuracy: 0.2119 - val_loss: 2.5602 - val_categorical_accuracy: 0.1842


23/23 [==============================] - 3s 123ms/step - loss: 2.3578 - categorical_accuracy: 0.2645 - val_loss: 2.4823 - val_categorical_accuracy: 0.2368
Epoch 196/800
23/23 [==============================] - 3s 113ms/step - loss: 2.3626 - categorical_accuracy: 0.2327 - val_loss: 2.4811 - val_categorical_accuracy: 0.2368
Epoch 197/800
23/23 [==============================] - 3s 117ms/step - loss: 2.3340 - categorical_accuracy: 0.2548 - val_loss: 2.4797 - val_categorical_accuracy: 0.2105
Epoch 198/800
23/23 [==============================] - 3s 117ms/step - loss: 2.3308 - categorical_accuracy: 0.2548 - val_loss: 2.4798 - val_categorical_accuracy: 0.2368
Epoch 199/800
23/23 [==============================] - 3s 115ms/step - loss: 2.3270 - categorical_accuracy: 0.2604 - val_loss: 2.4785 - val_categorical_accuracy: 0.2368
Epoch 200/800
23/23 [==============================] - 3s 115ms/step - loss: 2.3514 - categorical_accuracy: 0.2438 - val_loss: 2.4763 - val_categorical_accuracy: 0.2368


...

23/23 [==============================] - 3s 117ms/step - loss: 1.5505 - categorical_accuracy: 0.4958 - val_loss: 1.2474 - val_categorical_accuracy: 0.6842
Epoch 580/800
23/23 [==============================] - 3s 117ms/step - loss: 1.5662 - categorical_accuracy: 0.4723 - val_loss: 1.2584 - val_categorical_accuracy: 0.6842
Epoch 581/800
23/23 [==============================] - 3s 117ms/step - loss: 1.5679 - categorical_accuracy: 0.4931 - val_loss: 1.2518 - val_categorical_accuracy: 0.6842
Epoch 582/800
23/23 [==============================] - 3s 117ms/step - loss: 1.5635 - categorical_accuracy: 0.5000 - val_loss: 1.2373 - val_categorical_accuracy: 0.7105
Epoch 583/800
23/23 [==============================] - 3s 117ms/step - loss: 1.5608 - categorical_accuracy: 0.4765 - val_loss: 1.2409 - val_categorical_accuracy: 0.6842
Epoch 584/800
23/23 [==============================] - 3s 120ms/step - loss: 1.5577 - categorical_accuracy: 0.4972 - val_loss: 1.2359 - val_categorical_accuracy: 0.6842


23/23 [==============================] - 3s 116ms/step - loss: 1.4810 - categorical_accuracy: 0.5180 - val_loss: 1.1314 - val_categorical_accuracy: 0.7632
Epoch 628/800
23/23 [==============================] - 3s 119ms/step - loss: 1.4775 - categorical_accuracy: 0.5069 - val_loss: 1.1170 - val_categorical_accuracy: 0.7632
Epoch 629/800
23/23 [==============================] - 3s 116ms/step - loss: 1.4794 - categorical_accuracy: 0.5332 - val_loss: 1.1116 - val_categorical_accuracy: 0.7632
Epoch 630/800
23/23 [==============================] - 3s 117ms/step - loss: 1.4919 - categorical_accuracy: 0.5166 - val_loss: 1.1223 - val_categorical_accuracy: 0.7632
Epoch 631/800
23/23 [==============================] - 3s 126ms/step - loss: 1.4199 - categorical_accuracy: 0.5596 - val_loss: 1.1143 - val_categorical_accuracy: 0.7632
Epoch 632/800
23/23 [==============================] - 3s 120ms/step - loss: 1.4803 - categorical_accuracy: 0.5277 - val_loss: 1.1124 - val_categorical_accuracy: 0.7632


23/23 [==============================] - 3s 128ms/step - loss: 1.4141 - categorical_accuracy: 0.5706 - val_loss: 0.9997 - val_categorical_accuracy: 0.7895
Epoch 676/800
23/23 [==============================] - 3s 125ms/step - loss: 1.4076 - categorical_accuracy: 0.5360 - val_loss: 1.0164 - val_categorical_accuracy: 0.8158
Epoch 677/800
23/23 [==============================] - 3s 132ms/step - loss: 1.4369 - categorical_accuracy: 0.5499 - val_loss: 1.0200 - val_categorical_accuracy: 0.8421
Epoch 678/800
23/23 [==============================] - 3s 120ms/step - loss: 1.4498 - categorical_accuracy: 0.5388 - val_loss: 1.0031 - val_categorical_accuracy: 0.8421
Epoch 679/800
23/23 [==============================] - 3s 129ms/step - loss: 1.4376 - categorical_accuracy: 0.5319 - val_loss: 1.0049 - val_categorical_accuracy: 0.7895
Epoch 680/800
23/23 [==============================] - 3s 127ms/step - loss: 1.3756 - categorical_accuracy: 0.5720 - val_loss: 0.9989 - val_categorical_accuracy: 0.8158


23/23 [==============================] - 3s 124ms/step - loss: 1.3054 - categorical_accuracy: 0.5817 - val_loss: 0.8909 - val_categorical_accuracy: 0.8421
Epoch 724/800
23/23 [==============================] - 3s 124ms/step - loss: 1.3572 - categorical_accuracy: 0.5623 - val_loss: 0.8926 - val_categorical_accuracy: 0.8421
Epoch 725/800
23/23 [==============================] - 3s 126ms/step - loss: 1.3568 - categorical_accuracy: 0.5623 - val_loss: 0.8900 - val_categorical_accuracy: 0.8421
Epoch 726/800
23/23 [==============================] - 3s 127ms/step - loss: 1.2936 - categorical_accuracy: 0.5970 - val_loss: 0.8906 - val_categorical_accuracy: 0.8684
Epoch 727/800
23/23 [==============================] - 3s 123ms/step - loss: 1.3136 - categorical_accuracy: 0.5942 - val_loss: 0.8805 - val_categorical_accuracy: 0.8421
Epoch 728/800
23/23 [==============================] - 3s 119ms/step - loss: 1.3395 - categorical_accuracy: 0.5831 - val_loss: 0.8994 - val_categorical_accuracy: 0.8421


23/23 [==============================] - 4s 169ms/step - loss: 1.2660 - categorical_accuracy: 0.5748 - val_loss: 0.7951 - val_categorical_accuracy: 0.8684
Epoch 772/800
23/23 [==============================] - 3s 144ms/step - loss: 1.2629 - categorical_accuracy: 0.6260 - val_loss: 0.8034 - val_categorical_accuracy: 0.8684
Epoch 773/800
23/23 [==============================] - 3s 153ms/step - loss: 1.2570 - categorical_accuracy: 0.5762 - val_loss: 0.8043 - val_categorical_accuracy: 0.8684
Epoch 774/800
23/23 [==============================] - 4s 150ms/step - loss: 1.2799 - categorical_accuracy: 0.5873 - val_loss: 0.8082 - val_categorical_accuracy: 0.8684
Epoch 775/800
23/23 [==============================] - 3s 134ms/step - loss: 1.2664 - categorical_accuracy: 0.6053 - val_loss: 0.7959 - val_categorical_accuracy: 0.8684
Epoch 776/800
23/23 [==============================] - 4s 168ms/step - loss: 1.2416 - categorical_accuracy: 0.6108 - val_loss: 0.7883 - val_categorical_accuracy: 0.8684


In [158]:
y_pred = model.predict(X_test)

In [161]:
# Checking predictions manually
for pred, test in zip(y_pred, y_test):
    print(actions[np.argmax(pred)], actions[np.argmax(test)])

sleep sit
push push
eat eat
jump jump
wash wash
wait wait
fall give
climb climb
kick jump
climb climb
cry cry
sleep sleep
kick kick
fall fall
sit sit
wash sit
climb climb
kick kick
climb climb
eat eat
sleep sleep
drink drink
cry cry
drink drink
drink drink
kick kick
sleep sleep
cry cry
fall run
eat climb
walk wait
jump run
cry cry
cry cry
jump jump
look look
fall fall
climb climb
walk walk
climb climb


In [162]:
# Save model
model.save('action_16_classes.h5')

In [201]:
def prob_viz(res, actions, input_frame, colors):
    """That probability table on the side"""
    
    output_frame = input_frame.copy()
    
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60+num*25), (int(prob*100), 90+num*25), (245, 117, 16), -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        
    return output_frame

In [213]:
# Remember last 30 frames to predict
sequence = []

# Remember last 5 actions for stabilation
predictions = [0]

# Level of confidence
threeshold = 0.2

# Creating test case
import random
case = random.randint(0, len(label_map)-1)
test_case = [case]
for _ in range(4):
    while test_case[-1] == case:
        case = random.randint(0, len(label_map)-1)
    test_case.append(case)
print(test_case)
test_pass = True

# Start reading your face
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        if not test_case:
            break
        if test_pass:
            case = test_case.pop()
            test_pass = False
        
        ret, frame = cap.read()
        
        image, results = mediapipe_detection(frame, holistic)
        
        draw_styled_landmarks(image, results)
        
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        
#         Keep only last 30 frames
        sequence = sequence[-30:]
        
#         
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
        
#         Make sure that last 5 predictions same for stabilization
        if np.unique(predictions[-5:])[0] == np.argmax(res):
#             Max prediction is bigger than threeshold
            if res[np.argmax(res)] > threeshold:
                
#                 if test case passed pop it and new test
                if np.argmax(res) == case:
                    test_pass = True

#         put that side bar probability
        image = prob_viz(res, actions, image, colors)
            

#         How many of tests passed
        cv2.putText(image, actions[case], (540, 200),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        
        cv2.putText(image, f'{4-len(test_case)} of 4 is passed', (350, 90),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        
#         Putting tutorial image to right bottom
        tutorial_image = os.path.join(path_action_types, actions[case]+'.png')
        tutorial_image = cv2.imread(tutorial_image, cv2.IMREAD_COLOR)
        tutorial_image = cv2.resize(tutorial_image, (150, 120))
        h, w, _ = tutorial_image.shape

        image[360:h+360, 490:w+490] = tutorial_image
        
#         Show the video
        cv2.imshow('OpenCV Feed', image)
               
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

[10, 0, 6, 9, 13]


In [165]:
cap.release()
cv2.destroyAllWindows()